### Black Derman Toy 모형을 이용한 조정선도이자율 도출 모델

In [1]:
import numpy as np
import pandas as pd
import math

기본가정

In [2]:
# node 수
num_node = 5
# volatility
vol = 0.1
p = 0.5
up = np.exp(vol)
down = 1/up
# 만기액면금액
fv = 10000
#YTM
ytm = [0.05, 0.0551, 0.0604, 0.0659, 0.0715]
adj = np.zeros(num_node)
#채권 기간별 현재가치(YTM 할인)
pv = [fv/(1+ytm[i])**(i+1) for i in range(num_node)]


In [3]:
#조정전 선도이자율 tree 생성
spt = np.zeros((num_node,num_node))
for i in range(num_node):
    for j in range(num_node):
        if i == 0 and j == 0:
            spt[i,j] = ytm[0]
        elif spt[i,j] > 0:
            spt[i,j] = spt[i,j]
        else:
            if i>j:
                spt[i,j] = spt[i-1,j]*up
            elif i == j:
                spt[i,j] = spt[i-1, j-1] * down
            else:
                spt[i,j] = 0

pd.DataFrame(spt.T)

,0,1,2,3,4
0,0.05,0.055259,0.061070,0.067493,0.074591
1,0.00,0.045242,0.050000,0.055259,0.061070
2,0.00,0.000000,0.040937,0.045242,0.050000
3,0.00,0.000000,0.000000,0.037041,0.040937
4,0.00,0.000000,0.000000,0.000000,0.033516


기간별 채권가치 산정

In [4]:
def pvr(n):
    y = np.zeros((num_node+10,num_node+10,num_node+10))
    for i in range(num_node+1):
        for a in range(i+1,0,-1):
            for b in range(i+1,0,-1):
                c = i
                if int(a) == c+1:
                    y[a,b,c] = fv
                elif y[a,b,c] > 0:
                    y[a,b,c] = y[a,b,c]
                elif int(a) >= int(b):
                    y[a,b,c] = (y[a + 1, b, c] / (1 + spt[a-1, b-1] + adj[a - 1])) * p + (y[a + 1, b + 1, c] / (1 + spt[a-1, b-1] + adj[a - 1])) * (1 - p)
                else:
                    y[a,b,c] = 0
    return y[1,1,n]

조정선도이자율 도출을 위한 cost function

In [5]:
def cost(n):
    return pvr(n+1) - pv[n]

for i in range(num_node):
    print(cost(i))

0.0
85.5189526553986
246.29001524460364
470.1199239943526
740.063542988385


Gradient Descent

In [6]:
adj = np.zeros(num_node)
for i in range(1,num_node):
    tick = 0.0000000002
    k = 0
    while abs(cost(i)) > 0.01:
        k += 1
        d1 = cost(i)
        adj[i] += 0.000001
        d = (cost(i) - d1)/0.000001
        adj[i] -= 0.000001
        adj[i] -=  tick*d
        if k > 50000:
            break



결과 확인

In [7]:
# 조정선도이자율:
index = [str(i+1) + "년차" for i in range(num_node)]
print(pd.Series(adj, index = index))
adjspt = np.zeros((num_node,num_node))
for i in range(num_node):
    for j in range(num_node):
        if i>=j:
            adjspt[i,j] = spt[i,j] + adj[i]
pd.DataFrame(adjspt.T)

1년차    0.000000
2년차    0.009999
3년차    0.020648
4년차    0.031960
5년차    0.043427
dtype: float64


,0,1,2,3,4
0,0.05,0.065257,0.081718,0.099453,0.118018
1,0.00,0.055241,0.070648,0.087218,0.104497
2,0.00,0.000000,0.061585,0.077202,0.093427
3,0.00,0.000000,0.000000,0.069001,0.084364
4,0.00,0.000000,0.000000,0.000000,0.076943


In [8]:
#기간별 채권가치
adjpv = [pvr(i+1) for i in range(num_node)]
pd.Series(adjpv, index = index)

1년차    9523.809524
2년차    8982.817557
3년차    8386.701160
4년차    7747.020397
5년차    7080.097639
dtype: float64

In [9]:
#Cost Function 확인
for i in range(num_node):
    print(cost(i))

0.0
-0.0036145089361525606
0.006284629342189874
0.00978001353723812
0.002017991233515204
